In [ ]:
import os

import numpy as np
import pycolmap
import matplotlib.pyplot as plt
from tqdm import tqdm

from megadepth.utils.io import load_depth_map, load_image

In [ ]:
base_dir = "/Volumes/Extreme_SSD/MegaDepth/scenes/"
scene = "0229"

os.listdir(os.path.join(base_dir, scene, "sparse"))

In [ ]:
models = [
    "disk+sift+spsg",
    "superpoint_max-superglue-netvlad-50-KA+BA",
    "loftr-netvlad-50-KA+BA",
]

In [ ]:
# base_dir = os.path.join("..", "data", "scenes")

# model_name = "superpoint_max-superglue-netvlad-50-KA+BA"
model_name = models[-1]

model = pycolmap.Reconstruction(os.path.join(base_dir, scene, "sparse", model_name))
print(model.summary())

In [ ]:
def plot_depth_maps(image_name, models):
    fig, axes = plt.subplots(1, len(models) + 1, figsize=(5 * (len(models) + 1), 5))
    image = load_image(os.path.join(base_dir, scene, "images", image_name))
    axes[0].imshow(image)
    axes[0].set_title("Image")
    for i, model in enumerate(models):

        try:
            depth = load_depth_map(
                os.path.join(
                    base_dir,
                    scene,
                    "dense",
                    model,
                    "stereo/depth_maps",
                    f"{image_name}.geometric.bin",
                )
            )
        except FileNotFoundError:
            print(f"Depth map for {model} not found")
            continue

        depth = np.where(depth > 0, depth, np.nan)

        d_min = np.nanmin(depth)
        d_max = np.nanmax(depth)

        # normalize depth
        depth = (depth - d_min) / (d_max - d_min)

        axes[i + 1].imshow(depth)
        axes[i + 1].set_title(model)

    plt.show()

In [ ]:
img_names = [im.name for im in model.images.values()]
for img in np.random.choice(img_names, 10):
    plot_depth_maps(img, models)

In [ ]:
completeness = {}
for model in models:
    completeness[model] = []
    pbar = tqdm(img_names)
    for img in pbar:
        try:
            depth = load_depth_map(
                os.path.join(
                    base_dir, scene, "dense", model, "stereo/depth_maps", f"{img}.geometric.bin"
                )
            )
        except FileNotFoundError:
            continue

        comp = np.sum(depth > 0)
        completeness[model].append(comp / depth.size)

    pbar.close()


for model in models:
    print(f"{model}: {np.mean(completeness[model]):.4f}")

In [ ]:
fig, ax = plt.subplots(1, len(models), figsize=(5 * len(models), 5))

for i, model in enumerate(models):
    ax[i].hist(completeness[model], bins=20)
    ax[i].set_title(model)
    ax[i].set_xlim(0, 1)

# make all y-axes the same
max_y = max(ax[i].get_ylim()[1] for i in range(len(models)))
for i in range(0, len(models)):
    ax[i].set_ylim(0, max_y)

plt.show()